## Load image and export non-overlapping rectangular regions (ROIs) to new image files

### IMPORTS
Requires cv2, numpy, matplotlib

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
### OPTIONS

In [3]:
DEBUGGING          = False
doEXTERNALWINDOW   = False  # Don't make True if in GCP
doUSETESTIMAGERY   = False  # if False, use real images
doTESTCOLOR        = False

### Image path and filename
You need to define your inpath and filename (as infn) here.
This code is using a test image in a directory nearby that is defined as a relative path. 

In [4]:
inpath = "./../_IMAGERY/201004_MF_189_80_60_CB_S_1_360_64/"
infn = "201004_MF_189_80_60_CB_S_1_360_64.1+1.tif"
# infn = "201004_MF_189_80_60_CB_S_1_360_64.2+1.tif"

infile = inpath+infn
print( infile )


outpath = inpath+"REGIONS/"
print( outpath )
#if !exists(outpath):
#    os.mkdir( outpath)
outfile = outpath+infn
print(outfile)

./../_IMAGERY/201004_MF_189_80_60_CB_S_1_360_64/201004_MF_189_80_60_CB_S_1_360_64.1+1.tif
./../_IMAGERY/201004_MF_189_80_60_CB_S_1_360_64/REGIONS/
./../_IMAGERY/201004_MF_189_80_60_CB_S_1_360_64/REGIONS/201004_MF_189_80_60_CB_S_1_360_64.1+1.tif


### Read the image

In [5]:
img = cv2.imread( infile )

### Image shape
Shape of image is accessed by img.shape. It returns a tuple of number of rows, columns and channels (if image is color):

In [6]:
print(img.shape)

(23215, 14600, 3)


### Image size
Total number of pixels, from img.size

In [7]:
print(img.size)

1016817000


### Image datatype
Image datatype 

In [8]:
print(img.dtype)

uint8


### Display image using cv2.imshow (creates external window)
DON'T DO THIS IN A JUPYTER NOTEBOOK

In [9]:
# displaying image using cv2.imshow (creates external window)
# DON'T DO THIS IN A JUPYTER NOTEBOOK
if doEXTERNALWINDOW:
    cv2.imshow('image window for img', img )
    #add wait key. window waits till user press any key
    cv2.waitKey(0)

### Some recommend to destroy all windows, but this causes error:
cv2.destoyAllWindows()
AttributeError: module 'cv2' has no attribute 'destoyAllWindows'

### __________________________________________________________________
### Load and display image in the notebook using matplotlib
Use matplotlib instead of opencv to read and display the image

### Alternative: read the image in using matplotlib
imgplt = plt.imread( infile )
print(imgplt.shape)
print(imgplt.dtype)
plt.imshow(imgplt) # displays image in the next output cell of the jupyter notebook

### __________________________________________________________________
### Test iteration through image
Step through image by regions (ROIs, x,y blocks).
This will give you an idea how the code works, and demonstrate the process without saving the regions.

In [10]:
# TESTING 
ix      = 0
iy      = 0
iboxcount  = 0
intvpix = 200
NMAX    = 30

print( img.shape )
print( img.shape[0] )
print( img.shape[1] )

print('iboxcount, x, y' ) 
for x in range(     0, img.shape[0] , intvpix ):
    for y in range( 0, img.shape[1] , intvpix ):
        ix += 1
        iy += 1
        iboxcount += 1
        print( '%d %d %d (%d %d)' % (iboxcount, x, y, ix, iy) ) 
        print( img[x,y] )
         
        if iboxcount>NMAX:
            break
    if iboxcount>NMAX:
        break
   

(23215, 14600, 3)
23215
14600
iboxcount, x, y
1 0 0 (1 1)
[79 79 79]
2 0 200 (2 2)
[205 205 205]
3 0 400 (3 3)
[193 193 193]
4 0 600 (4 4)
[193 193 193]
5 0 800 (5 5)
[215 215 215]
6 0 1000 (6 6)
[213 213 213]
7 0 1200 (7 7)
[187 187 187]
8 0 1400 (8 8)
[204 204 204]
9 0 1600 (9 9)
[206 206 206]
10 0 1800 (10 10)
[154 154 154]
11 0 2000 (11 11)
[151 151 151]
12 0 2200 (12 12)
[156 156 156]
13 0 2400 (13 13)
[176 176 176]
14 0 2600 (14 14)
[123 123 123]
15 0 2800 (15 15)
[133 133 133]
16 0 3000 (16 16)
[122 122 122]
17 0 3200 (17 17)
[182 182 182]
18 0 3400 (18 18)
[150 150 150]
19 0 3600 (19 19)
[133 133 133]
20 0 3800 (20 20)
[138 138 138]
21 0 4000 (21 21)
[153 153 153]
22 0 4200 (22 22)
[162 162 162]
23 0 4400 (23 23)
[160 160 160]
24 0 4600 (24 24)
[163 163 163]
25 0 4800 (25 25)
[162 162 162]
26 0 5000 (26 26)
[158 158 158]
27 0 5200 (27 27)
[98 98 98]
28 0 5400 (28 28)
[71 71 71]
29 0 5600 (29 29)
[32 32 32]
30 0 5800 (30 30)
[46 46 46]
31 0 6000 (31 31)
[68 68 68]


### __________________________________________________________________
## Export regions to images
This is the code that iterates through image by regions (ROIs, x,y blocks), 
and saves the regions to new image files. 

In [12]:
# Export regions to images 
       
ix = 0
iy = 0
icount = 0
boxx    = 2000
boxy    = 2000
NROIMAX = 444

print( img.shape[0] )
print( 'boxx init = %d' % boxx )    
print( boxx )

if img.shape[0] <= boxx:
    boxx    = 100
    boxy    = 100
    NROIMAX =  10

print( 'boxx now = %d' % boxx )    
    
print('icount, x, y' ) 
#for x in range(     0, img.shape[0] - boxx , boxx ):
#    for y in range( 0, img.shape[1] - boxy , boxy ):
for x in range(     0, img.shape[0]  , boxx ):
    for y in range( 0, img.shape[1]  , boxy ):  
        ix += 1
        iy += 1
        icount += 1
        print('%d %d %d' % (icount, x, y) ) 
        print( img[x,y] )
        
        # region of interest rectangle selection
        ROI = img[ x:x+boxx , y:y+boxy ]
        
        # outfilename
        # ofnpfx = infile+'_{}.png'
        ofnpfx = outfile+'_{}.png'
        ofn = ofnpfx.format( icount , width=5 )   # apparently the width option is not working here     
        print( ofn )
        
        # write region contents to image file (and works w/ color image content)
        cv2.imwrite( ofn , ROI)
        
        if icount>NROIMAX:
            break
    if icount>NROIMAX:
        break
 

23215
boxx init = 2000
2000
boxx now = 2000
icount, x, y
1 0 0
[79 79 79]
./../_IMAGERY/201004_MF_189_80_60_CB_S_1_360_64/REGIONS/201004_MF_189_80_60_CB_S_1_360_64.1+1.tif_{}.png
./../_IMAGERY/201004_MF_189_80_60_CB_S_1_360_64/REGIONS/201004_MF_189_80_60_CB_S_1_360_64.1+1.tif_1.png
2 0 2000
[151 151 151]
./../_IMAGERY/201004_MF_189_80_60_CB_S_1_360_64/REGIONS/201004_MF_189_80_60_CB_S_1_360_64.1+1.tif_{}.png
./../_IMAGERY/201004_MF_189_80_60_CB_S_1_360_64/REGIONS/201004_MF_189_80_60_CB_S_1_360_64.1+1.tif_2.png
3 0 4000
[153 153 153]
./../_IMAGERY/201004_MF_189_80_60_CB_S_1_360_64/REGIONS/201004_MF_189_80_60_CB_S_1_360_64.1+1.tif_{}.png
./../_IMAGERY/201004_MF_189_80_60_CB_S_1_360_64/REGIONS/201004_MF_189_80_60_CB_S_1_360_64.1+1.tif_3.png
4 0 6000
[68 68 68]
./../_IMAGERY/201004_MF_189_80_60_CB_S_1_360_64/REGIONS/201004_MF_189_80_60_CB_S_1_360_64.1+1.tif_{}.png
./../_IMAGERY/201004_MF_189_80_60_CB_S_1_360_64/REGIONS/201004_MF_189_80_60_CB_S_1_360_64.1+1.tif_4.png
5 0 8000
[28 28 28]
./..

In [13]:
print('N images = %d' % icount) 

N images = 96


Load and display image in the notebook using matplotlib

Use matplotlib instead of opencv to read and display the image


In [102]:
### Alternative: read the image in using matplotlib
infn = "./block/201004_MF_189_80_60_CB_S_1_360_64.1+1.tif_1.png"
infile = inpath+infn
print( infile )

./../_IMAGERY/201004_MF_189_80_60_CB_S_1_360_64/./block/201004_MF_189_80_60_CB_S_1_360_64.1+1.tif_1.png


In [ ]:
imgplt = plt.imread( infile )

print(imgplt.shape)

plt.imshow(imgplt) # displays image in the next output cell of the jupyter notebook